In [1]:
import os
import pandas as pd
import numpy as np
import json
from utils import nlp

In [2]:
special_values = ['none', 'yes', 'no', 'dontcare', '?']

In [16]:
feats_lstm = json.load(open('../../sc-lstm/resource/mwoz/feat.json'))
data_lstm = json.load(open('../../sc-lstm/resource/mwoz/text.json'))
split_list = json.load(open('../../sc-lstm/resource/mwoz/data_split/allDataSplitRand0925.json'))

In [17]:
test_list = [elem[0] for elem in split_list['test']]
test_list = list(set(test_list))

In [18]:
feats = {}
for elem in test_list:
    feats[elem] = feats_lstm[elem]

In [19]:
new_feat = {}
for key, dialog in feats.items():
    new_feats_dac = {}
    for k1, ft in dialog.items():
        lst_feat_diag = {}
        for k, v in ft.items():
            for val in v:
                new_key = k.lower()+'-'+val[0].lower()+'-'+val[1]
                lst_feat_diag[new_key] = val[2]
        new_feats_dac[k1] = lst_feat_diag
    new_feat[key] = new_feats_dac

In [29]:
new_feat

{'PMUL1344': {'7': {'general-welcome-none-none': 'none'},
  '1': {'attraction-recommend-type-1': 'museum ',
   'attraction-request-price-?': '?',
   'attraction-nooffer-area-1': 'the centre',
   'attraction-nooffer-type-1': 'entertainment '},
  '5': {'train-inform-leave-1': '5:35'},
  '8': {'general-bye-none-none': 'none'},
  '4': {'train-request-depart-?': '?',
   'train-inform-choice-1': '5',
   'train-inform-depart-1': 'london kings cross'},
  '3': {'general-reqmore-none-none': 'none',
   'attraction-inform-fee-1': 'no entrance fee',
   'attraction-inform-post-1': 'cb23bu',
   'attraction-inform-phone-1': '01223334900',
   'attraction-recommend-name-1': "Christ's college"},
  '6': {'train-offerbooked-ticket-1': '17.6 GBP',
   'train-offerbooked-ref-1': 'WOG8WD79'},
  '2': {'attraction-inform-type-1': 'college',
   'attraction-inform-choice-1': '13'}},
 'PMUL4564': {'1': {'attraction-request-area-?': '?',
   'attraction-inform-type-1': 'entertainment',
   'attraction-inform-choice-1'

In [ ]:
feats['PMUL4504']

In [ ]:
lst = []
for k1, v1 in data_lstm.items():
    for k2, v2 in v1.items():
        try:
            lst.append((k1, k2, nlp.normalize(v2['delex'])))
        except:
            pass

# Results sc-lstm

In [3]:
fin = open('sclstm_lstm.res')
results_lstm = fin.readlines()

In [4]:
targets = [elem.split('Target:')[-1].strip() for elem in results_lstm if elem.startswith('Target')]
gens = [elem.split(':')[-1].strip() for elem in results_lstm if elem.startswith('Gen')]
tg_lst = []
for idx, elem in enumerate(gens):
    tg_lst.append((targets[idx], elem))

In [9]:
import random
random.seed(33)
len(random.sample(gens, 7300))

7300

In [7]:
mini = ['yes it does . can i book it for you ?','is there a time you need to arrive by ?']

In [10]:
sampled = random.sample(gens, 7300)
sampled = sampled+(mini*36)
random.shuffle(sampled)

In [11]:
len(sampled)

7372

# Lexicalization

In [12]:
import os

In [13]:
fin = open('template_custom.txt')
template = fin.readlines()

In [14]:
template = [elem.strip() for elem in template]
len(template)

192

In [20]:
my_lst = []
for k, v in new_feat.items():
    for k1, v1 in v.items():
        my_lst.append(v1)

In [21]:
lexicalizer = {}
for elem in my_lst:
    for k, v in elem.items():
        if v != 'none' and v is not '?':
            lexicalizer['slot-'+k[:-2]] = v

In [22]:
len(lexicalizer)

145

In [23]:
res = []
for utt in sampled:
    for key, val in lexicalizer.items():
        utt = utt.replace(key, val)
    res.append((utt, '0'))

In [24]:
len(res)

7372

In [25]:
res

[('yes i have booked your stay for 2 nights starting monday . your stay is JTWXFM7M .',
  '0'),
 ('it is a moderate 4 star hotels in the  east .', '0'),
 ('i found five restaurants in the  west part of town and is  expensive . would you like to try a different area ?',
  '0'),
 ('Saint Johns Chop House is in the  west . the postcode is cb11bg .', '0'),
 ('i would recommend Cambridge Artworks , the entrance fee is no entrance fee and the post code is cb13ef . may i help with something else ?',
  '0'),
 ('arbury lodge guesthouse is located in the  east . would that work ?', '0'),
 ("i 'm sorry . there are five  expensive restaurants in the  east . there are five in the  west and five in the  west part of town . do you have an preference ?",
  '0'),
 ('there is the  east and arbury lodge guesthouse is a 4 star hotels in the  east , and  east . do you have a preference ?',
  '0'),
 ("arbury lodge guesthouse is a 4 star rating , they are located in the  east , it is a moderate price range .

In [26]:
df = pd.DataFrame(res, columns=['text','label'])
df = df.sample(frac=1)
df.to_csv('test_bert_sc_lstm_new.csv', index=False)